<a href="https://colab.research.google.com/github/IshanAK/UtiliQue/blob/cost_prediction/cost_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing all necessary libraries to work with the dataset
import numpy as np # linear algebra
#Using Matplotlib library it able to plot all types 2D graphs in python
import matplotlib.pyplot as plt 
#sklearn.metrics includes score functions, performance metrics and pairwise metrics and distance computations.
import sklearn.metrics as sm
# Using Panda library it able to get a dataframe with rows and columns out of the dataset
import pandas as pd
from sklearn import linear_model
%matplotlib inline
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
data = pd.read_csv("/content/drive/My Drive/CDAP/Dataset/dataset.csv")
data.head()

,Distance(Km),Cost for Distance,Working Hours,Cost for Working Hours,Total,Priority Level,Cost for Priority,Net Total
0,0,0,2,1000,1000,0,0,1000
1,3,150,1,500,650,0,0,650
2,4,200,4,2000,2200,0,0,2200
3,1,50,3,1500,1550,1,155,1705
4,2,100,3,1500,1600,0,0,1600


In [ ]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Distance(Km),9999.0,2.479648,1.716552,0.0,1.0,3.0,4.0,5.0
Cost for Distance,9999.0,123.982398,85.827591,0.0,50.0,150.0,200.0,250.0
Working Hours,9999.0,3.007501,1.414158,1.0,2.0,3.0,4.0,5.0
Cost for Working Hours,9999.0,1503.750375,707.079153,500.0,1000.0,1500.0,2000.0,2500.0
Total,9999.0,1627.732773,711.320719,500.0,1050.0,1650.0,2200.0,2750.0
Priority Level,9999.0,0.998300,0.815984,0.0,0.0,1.0,2.0,2.0
Cost for Priority,9999.0,162.637764,161.451010,0.0,0.0,130.0,255.0,550.0
Net Total,9999.0,1790.370537,795.578993,500.0,1150.0,1800.0,2475.0,3300.0


In [ ]:
#Check for existing null values
total_null_values = sum(data.isnull().sum())
print(total_null_values)

0


In [ ]:
#LabelEncoder use to normalize labels which they contain only values between 0 and n_classes-1. 
from sklearn.preprocessing import LabelEncoder
Enc = LabelEncoder()
data_tf = data.copy()
for i in data.columns:
    data_tf[i]=Enc.fit_transform(data[i])

In [ ]:
#Drop the 'Outcome' column which represent the feacture which we are going to predict 
X = data_tf.drop(['Net Total'], axis=1)
#assign 'Outcome' column to Y varible
Y = data_tf['Net Total']
feature_list = list(X.columns)

In [ ]:
#Dataset has been separated by 80/20 for training and testing.
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 42)

print(X_train.shape,' is the shape of Training Features')
print(Y_train.shape,' is the shape of Training Lables')
print(X_test.shape,' is the shape of Testing Features')
print(Y_train.shape,' is the shape of Testing Lables')

(7999, 7)  is the shape of Training Features
(7999,)  is the shape of Training Lables
(2000, 7)  is the shape of Testing Features
(7999,)  is the shape of Testing Lables


In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
pickle.dump(reg, open('model.pkl','wb'))

In [ ]:
reg.coef_

array([-0.36381209, -0.36381209,  0.44173964,  0.44173964,  2.28662574,
        1.63205255,  0.18161584])

In [ ]:
reg.predict([[0,0,2,1,1,0,0]])

array([3.15877281])

In [ ]:
#Training accuracy of the model 
reg.score(X_train, Y_train)

0.9981408849465556

In [ ]:
#Impliment the RandomForestClassifier 
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=10)
rnd_clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#Do the predicitions using test data
Y_pred = rnd_clf.predict(X_test)
Y_pred

array([65, 47,  1, ...,  5, 66, 60])

In [ ]:
#Get the accuracy score
accuracyscore=sm.accuracy_score(Y_test, Y_pred)
print('Accuracy score given for test data:',str(accuracyscore))

Accuracy score given for test data: 1.0


In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test, rnd_clf.predict(X_test))
  
TN = cm[0][0]
TP = cm[1][1]
FN = cm[1][0]
FP = cm[0][1]
  
print(cm)
print('Model[{}] Testing Accuracy = "{}!"'.format(0,  (TP + TN) / (TP + TN + FN + FP)))
print()# Print a new line

[[18  0  0 ...  0  0  0]
 [ 0 38  0 ...  0  0  0]
 [ 0  0 44 ...  0  0  0]
 ...
 [ 0  0  0 ... 27  0  0]
 [ 0  0  0 ...  0 26  0]
 [ 0  0  0 ...  0  0 28]]
Model[0] Testing Accuracy = "1.0!"



In [ ]:
#Classification report
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


#Check precision, recall, f1-score
print('Classification report:')
print()# Print a new line
print( classification_report(Y_test, rnd_clf.predict(X_test)) )
#Another way to get the models accuracy on the test data
print( accuracy_score(Y_test, rnd_clf.predict(X_test)))
print()#Print a new line

Classification report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        38
           2       1.00      1.00      1.00        44
           3       1.00      1.00      1.00        22
           4       1.00      1.00      1.00        23
           5       1.00      1.00      1.00        49
           6       1.00      1.00      1.00        20
           7       1.00      1.00      1.00        28
           8       1.00      1.00      1.00        14
           9       1.00      1.00      1.00        12
          10       1.00      1.00      1.00        26
          11       1.00      1.00      1.00        18
          12       1.00      1.00      1.00        22
          13       1.00      1.00      1.00        20
          14       1.00      1.00      1.00        17
          15       1.00      1.00      1.00        24
          16       1.00      1.00      1.00        27
   